In [13]:
# imports
import os 
import glob 
from dotenv import load_dotenv
!pip install gradio
import gradio as gr
import glob



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [9]:
# imports for langchain
!pip install -U langchain-community    #upgrade langchain ti the latest available version
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [10]:
#low cost model
model = "gpt-4o-mini"
db_name = "vector_db"

In [12]:
# load environment variable to a file called .env

load_dotenv()   #load environment variable from .env file to our python environment
openai_api_key = os.getenv("OPENAI_API_KEY")


In [19]:
#read in each documents using langchain loaders
# take everthing from all our sub-folders of our knowledge base

folders = glob.glob("knowledge_base/*")
print(folders)

['knowledge_base/company', 'knowledge_base/contracts', 'knowledge_base/products', 'knowledge_base/employee']


In [27]:
#best practice to read all files in folders


# for folder in folders:
#     files = glob.glob(os.path.join(folder, "*.txt"))
#     for file in files:
#         with open(file, 'r') as f:
#             content = f.read()
#             # print(content)

In [46]:
folders = glob.glob("knowledge_base/*")
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    # print(doc_type)
    loader = DirectoryLoader(folder, glob="**/*.txt", loader_cls=TextLoader)  #from this folder load all .txt files even in subfolders and load them all with TextLoader class
    folder_docs = loader.load()
    # print(folder_docs)
    for doc in folder_docs:
        doc.metadata["doc type"] = doc_type
        # print(f"doc:{doc}")
        documents.append(doc)
print(len(documents))
    

31


In [51]:
documents[30]

Document(metadata={'source': 'knowledge_base/employee/Maxine Thompson.txt', 'doc type': 'employee'}, page_content="# HR Record\n\n# Maxine Thompson\n\n## Summary\n- **Date of Birth:** January 15, 1991  \n- **Job Title:** Data Engineer  \n- **Location:** Austin, Texas  \n\n## Insurellm Career Progression\n- **January 2017 - October 2018**: **Junior Data Engineer**  \n  * Maxine joined Insurellm as a Junior Data Engineer, focusing primarily on ETL processes and data integration tasks. She quickly learned Insurellm's data architecture, collaborating with other team members to streamline data workflows.  \n- **November 2018 - December 2020**: **Data Engineer**  \n  * In her new role, Maxine expanded her responsibilities to include designing comprehensive data models and improving data quality measures. Though she excelled in technical skills, communication issues with non-technical teams led to some project delays.  \n- **January 2021 - Present**: **Senior Data Engineer**  \n  * Maxine was

In [55]:
textsplitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  #overlaps help preserve context between chunks
chunks = textsplitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [56]:
len(chunks)

122

In [58]:
chunks[100]

Document(metadata={'source': 'knowledge_base/employee/Jordan K. Bishop.txt', 'doc type': 'employee'}, page_content="# HR Record\n\n# Jordan K. Bishop\n\n## Summary\n- **Date of Birth:** March 15, 1990\n- **Job Title:** Frontend Software Engineer\n- **Location:** Austin, Texas\n\n## Insurellm Career Progression\n- **June 2018:** Hired as a Frontend Software Engineer.\n- **August 2019:** Promoted to Senior Frontend Software Engineer due to outstanding contributions to the Insurellm web application redesign project.\n- **March 2021:** Led a cross-functional team for the launch of Insurellm's customer portal, enhancing user experience and engagement.\n- **January 2022:** Transitioned to a mentorship role, where Jordan K. Bishop began training junior engineers, which affected the focus on personal projects.\n- **August 2023:** Returned to core development tasks but faced challenges adapting to new frameworks, leading to performance reviews reflecting a need for improvement.")

In [64]:
# we want to be sure all chunks has its own metadata
doc_types = set(chunk.metadata['doc type'] for chunk in chunks)
print(f"documnet types  found:{', '.join(doc_types)}")

documnet types  found:company, contracts, employee, products


In [70]:
for chunk in chunks:
    if 'CEO'in chunk.page_content:
        print(chunk)
        print("-"*500)
        

page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, available 24/7.  
2. **Technical Maintenance**: Regular maintenance and updates to the Carllm platform will be conducted by Insurellm, with any downtime communicated in advance.  
3. **Training & Resources**: Initial training sessions will be provided for Velocity Auto Solutions’ staff to ensure effective use of the Carllm suite. Regular resources and documentation will be made available online.

---

**Accepted and Agreed:**  
**For Velocity Auto Solutions**  
Signature: _____________________  
Name: John Doe  
Title: CEO  
Date: _____________________  

**For Insurellm**  
Signature: _____________________  
Name: Jane Smith  
Title: VP of Sales  
Date: _____________________' metadata={'source': 'knowledge_base/contracts/Contract with Velocity Auto Solutions for Carllm.txt', 'doc type': 'contracts'}
-------------------------------------